In [1]:
import math
import ipyleaflet
from ipyleaflet import *
import json
import pandas as pd
import requests
import geopandas
import ipywidgets as widgets
from ipywidgets import *
from branca.colormap import linear
from datetime import datetime
from IPython.display import clear_output

In [2]:
# import the dataset as a pandas dataframe
df = pd.read_csv("fire.csv")

In [3]:
'''
because the dataset does not have a state name column, this process creates a state column 
using the FIRE_ID so that we are able to create a choropleth
'''
state_names = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

state_list = []
for (index, row) in df.iterrows():
    state_list.append(row.loc['FIRE_ID'][:2])
df['STATE'] = state_list
df = df[df.STATE != "PR"]


In [4]:
# deleting unecessary columns
del df['PRE_ID']
del df['POST_ID']
del df['NODATA_THRESHOLD']
del df['LOW_THRESHOLD']
del df['MODERATE_THRESHOLD']
del df['HIGH_THRESHOLD']
del df['GREENNESS_THRESHOLD']

In [5]:
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
    
geo_json_data = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/us-states.json',
    'us-states.json',
     json.load)

# this function allows us to visualize the size of the fire by converting acres to radius
# this is an estimation as fires do not spread in perfect circles, however the size of area affected is accurate
def acres_to_radius(a):
    sq_meter = int(a) * 4046.86
    radius = math.sqrt(sq_meter/math.pi)
    return int(radius)



# because the dataset uses a timestamp format, if we want to filter by month, we must convert it to the standard format
# we add columns for the month, year, and day of the fire so that it is easier to understand by looking at the dataframe
# this also makes the date easier to work with through interaction when filtering
year = []
months = []
days = []
for (index, row) in df.iterrows():
    this_year = datetime.fromtimestamp(int(row.loc['IG_DATE'])/1000).year
    this_month = datetime.fromtimestamp(int(row.loc['IG_DATE'])/1000).month
    this_day = datetime.fromtimestamp(int(row.loc['IG_DATE'])/1000).day
    year.append(this_year)
    months.append(this_month)
    days.append(this_day)
df['MONTH'] = months
df['DAY'] = days
df['YEAR'] = year

In [6]:
# this begins our filtering process. For example, if we do not wish to filter by month, we can set this value to ALL
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

# this function creates and returns a choropleth map given a filtered dataframe
def create_choro(df):
    cho_data = df['STATE'].value_counts().to_dict()
    for state in state_names:
        keys = list(cho_data.keys())
        if state not in keys:
            cho_data[state] = 0
    choro = ipyleaflet.Choropleth(
        name="Choropleth",
        geo_data = geo_json_data,
        choro_data = cho_data,
        colormap=linear.YlOrRd_07,
        border_color='black',
        style={'fillOpacity': 0.7, 'dashArray': '3, 3'}
    )
    choro_map = ipyleaflet.Map(zoom=4, center = (39,-100))
    choro_map.add_layer(choro)
    return choro_map

# this function allows us to export our filtered dataframe as a CSV file
def export(a):
    new_df = df.copy(deep=True)
    if dropdown_state.value != ALL:
        new_df = new_df[new_df.STATE == dropdown_state.value]
    if dropdown_type.value != ALL:
        new_df = new_df[new_df.FIRE_TYPE == dropdown_type.value]
    if dropdown_month.value != ALL:
        new_df = new_df[new_df.MONTH == dropdown_month.value]
    new_df.to_csv(r"Matches.csv")
    print("Exported CSV as 'Matches.csv'")

# this function dictates what happens when the generate map button is clicked
def clicked(b):
#   I put clear output so that the previous maps generated are cleared
    clear_output()
#   because the previous output was cleared, we must once again display all of our filters and our generate map button
    display(dropdown_state)
    display(dropdown_month)
    display(dropdown_type)
    display(show_matches)
    display(check_choro)
    display(generate) 
#     a new map is created
    new_map = ipyleaflet.Map(zoom=4, center = (39,-100))
#     we copy the orignal dataframe so we can apply the selected filters without deleting data
    new_df = df.copy(deep=True)
#     each of these if statements filters the data so that only the selected value in the dropdown remains in the dataset
#     the filters check the value selected and exclude all other data
    if dropdown_state.value != ALL:
        new_df = new_df[new_df.STATE == dropdown_state.value]
    if dropdown_type.value != ALL:
        new_df = new_df[new_df.FIRE_TYPE == dropdown_type.value]
    if dropdown_month.value != ALL:
        new_df = new_df[new_df.MONTH == dropdown_month.value]
#   at this point the data is filtered
#   this filter checks if the user wants a choropleth, if so one is created and the new map is set to the created choropleth and displayed
    if check_choro.value == True:
        new_map = create_choro(new_df)
        display(new_map)
#       the choropleth checks if the user wishes to display the details of the fire
        if show_matches.value:
            display(new_df)
#       this return statements stops the function from further adding points to the map as the choropleth has already been generated
        return
#   Now, since the data is filtered and the user has chosen not to view a choropleth, points are added to the map one by one.
    for (index, row) in new_df.iterrows():
#       this marker allows for the user to hover over the different fires and see their names, type, and area.   
        marker = Marker(location=(row.loc['LATITUDE'], row.loc['LONGITUDE']), title=row.loc['FIRE_NAME'] + ": " + row.loc['FIRE_TYPE'] + "; " + str(row.loc['ACRES']) + " Acres", visible=False, draggable=False)
#       The fucntion checks which type of fire is being plotted so that it may be color coded.
        if row.loc['FIRE_TYPE'] == "Wildfire":
#           when the circle is added, the radius is determed by the above function "acres to radius". The resulting circle has the same area specified by the dataset  
            circle = Circle(location=[row.loc['LATITUDE'], row.loc['LONGITUDE']], radius=acres_to_radius(row.loc['ACRES']), fill_color="#FF0000", fill_opacity=.75, stroke=False)
        elif row.loc['FIRE_TYPE'] == "Prescribed Fire":
            circle = Circle(location=[row.loc['LATITUDE'], row.loc['LONGITUDE']], radius=acres_to_radius(row.loc['ACRES']), fill_color="#FFA500", fill_opacity=.75, stroke=False)
        elif row.loc['FIRE_TYPE'] == "Unknown":
            circle = Circle(location=[row.loc['LATITUDE'], row.loc['LONGITUDE']], radius=acres_to_radius(row.loc['ACRES']), fill_color="#000000", fill_opacity=.75, stroke=False)
#       the circles and labels are added to the map  
        new_map.add_layer(circle)
        new_map.add_layer(marker)

#   the map with all of the filtered points is now displayed
    display(new_map)
#   if the user chose to show the details of the matches, the details are displayed, and the export as csv option is now available
    if show_matches.value:
        display(new_df)
        print(len(new_df))
        display(export_csv)
        
    

export_csv = widgets.Button(description="Export as CSV")
export_csv.on_click(export)
generate = widgets.Button(description="Generate Map")
generate.on_click(clicked)
dropdown_month = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.MONTH), description = "Month")
dropdown_type = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.FIRE_TYPE), description = "Fire Type")
dropdown_state = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.STATE), description ="State")
show_matches = widgets.Checkbox(
    value=False,
    description='View Match Detail',
    disabled=False,
    indent=True
)
check_choro = widgets.Checkbox(
    value=False,
    description='Choropleth',
    disabled=False,
    indent=True
)

display(dropdown_state)
display(dropdown_month)
display(dropdown_type)
display(show_matches)
display(check_choro)
display(generate)

Dropdown(description='State', index=7, options=('ALL', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'FL', 'GA', 'ID', '…

Dropdown(description='Month', options=('ALL', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value='ALL')

Dropdown(description='Fire Type', options=('ALL', 'Prescribed Fire', 'Unknown', 'Wildfire'), value='ALL')

Checkbox(value=False, description='View Match Detail')

Checkbox(value=False, description='Choropleth')

Button(description='Generate Map', style=ButtonStyle())

Map(center=[39, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…